In [2]:
#import dependencies
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import psycopg2

In [3]:
#read the csv
hotel_df = pd.read_csv('hotel_data.csv')
hotel_df.head()

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,Good location on the Lido.,NaN,Julie,NaN
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,������ ���������������,NaN,sungchul,NaN


Clean the CSV

In [4]:
reduced_hotel_df = hotel_df.loc[:, ["city","name","reviews.date","reviews.rating"]]
reduced_hotel_df.head()

,city,name,reviews.date,reviews.rating
0,Mableton,Hotel Russo Palace,2013-09-22T00:00:00Z,4.0
1,Mableton,Hotel Russo Palace,2015-04-03T00:00:00Z,5.0
2,Mableton,Hotel Russo Palace,2014-05-13T00:00:00Z,5.0
3,Mableton,Hotel Russo Palace,2013-10-27T00:00:00Z,5.0
4,Mableton,Hotel Russo Palace,2015-03-05T00:00:00Z,5.0


In [5]:
replace_zero_df = reduced_hotel_df.replace(0, np.nan)
hotel_drop_rating_na_df = replace_zero_df.dropna(subset = ['reviews.rating'])
hotel_drop_rating_na_df.head()

,city,name,reviews.date,reviews.rating
0,Mableton,Hotel Russo Palace,2013-09-22T00:00:00Z,4.0
1,Mableton,Hotel Russo Palace,2015-04-03T00:00:00Z,5.0
2,Mableton,Hotel Russo Palace,2014-05-13T00:00:00Z,5.0
3,Mableton,Hotel Russo Palace,2013-10-27T00:00:00Z,5.0
4,Mableton,Hotel Russo Palace,2015-03-05T00:00:00Z,5.0


In [7]:
average_rating_df = hotel_drop_rating_na_df.groupby('name').aggregate({'reviews.rating': 'mean'}).reset_index()
average_rating_df = average_rating_df.rename(columns={'reviews.rating':'Average Ratings'})
average_rating_df

,name,Average Ratings
0,1785 Inn,3.500000
1,1900 House,4.923077
2,40 Berkeley Hostel,3.329193
3,A Bed & Breakfast In Cambridge,3.574074
4,Acorn Motor Inn,3.750000
...,...,...
626,Wingate By Wyndham Pueblo,4.433333
627,Wisconsin-aire Motel,4.285714
628,Wyndham Resort At Fairfield Mountains,3.928571
629,Wyndham Vacation Resorts Towers On The Grove,4.050000


In [13]:
agg_df = hotel_drop_rating_na_df.merge(average_rating_df)
agg_df = agg_df.rename(columns={'name':'hotel_name', 'reviews.date':"date_of_review", 'reviews.rating':'hotel_rating', 'Average Ratings':'average_rating'})
agg_df.head()

,city,hotel_name,date_of_review,hotel_rating,average_rating
0,Mableton,Hotel Russo Palace,2013-09-22T00:00:00Z,4.0,4.052632
1,Mableton,Hotel Russo Palace,2015-04-03T00:00:00Z,5.0,4.052632
2,Mableton,Hotel Russo Palace,2014-05-13T00:00:00Z,5.0,4.052632
3,Mableton,Hotel Russo Palace,2013-10-27T00:00:00Z,5.0,4.052632
4,Mableton,Hotel Russo Palace,2015-03-05T00:00:00Z,5.0,4.052632


In [15]:
#connect to local database
rds_connection_string = "postgres:postgres@localhost:5432/hotels_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [16]:
#check table names
engine.table_names()

['csv']

In [17]:
#load dataframe into database
agg_df.to_sql(name='csv', con=engine, if_exists='append', index=False)

In [18]:
#confirm that dataframe has been properly loaded by querying table
pd.read_sql_query('select * from csv', con=engine).head()

,index,city,hotel_name,date_of_review,hotel_rating,average_rating
0,1,Mableton,Hotel Russo Palace,2013-09-22,4.0,4.052632
1,2,Mableton,Hotel Russo Palace,2015-04-03,5.0,4.052632
2,3,Mableton,Hotel Russo Palace,2014-05-13,5.0,4.052632
3,4,Mableton,Hotel Russo Palace,2013-10-27,5.0,4.052632
4,5,Mableton,Hotel Russo Palace,2015-03-05,5.0,4.052632
